# Total unimodularity

## Introduction to optimization and operations research.

Michel Bierlaire


In [ ]:

import numpy as np


In this lab, you will implement tools to check **total unimodularity (TU)** of a matrix.
You will (i) verify entries are in {−1, 0, 1} with numerical tolerance, (ii) compute determinants
of square submatrices via a recursive cofactor expansion with early stopping, and (iii) raise
informative exceptions when a determinant falls outside {−1, 0, 1}. This hands‑on approach
clarifies why TU matters: if the constraint matrix of a **linear optimization problem** is TU
(and the right‑hand side is integral), then all extreme points are integral—so solving the
continuous relaxation already yields integer solutions. These exercises connect numerical checks,
mathematical definitions, and strong modeling implications in network, flow, and assignment problems.

Write a Python function that verifies if an element is 0, -1, or 1.

In [ ]:
def check_valid_values(the_value: float) -> bool:
    """Check if an element is 0, -1, or 1.

    :param the_value: value to check
    :return: True if it is 0, -1 or 1, False otherwise.
    """
    # Due to numerical imprecision in finite arithmetic, values such as 1.0e-18 should be considered valid.
    # Use the numpy function `isclose` to perform the comparisons.
    ...









Let's test the function.

Expected output: `True` for the following cases.

In [ ]:
print(check_valid_values(the_value=0.0))

In [ ]:
print(check_valid_values(the_value=1.0))

In [ ]:
print(check_valid_values(the_value=-1.0))

In [ ]:
print(check_valid_values(the_value=1.0e-18))

Expected output: `False` for the following cases.

In [ ]:
print(check_valid_values(the_value=0.01))

In [ ]:
print(check_valid_values(the_value=3.4))

In [ ]:
print(check_valid_values(the_value=-2))

Write a Python function that calculates a determinant recursively using the cofactor expansion rule.

<div style="font-family: Arial, sans-serif; color: darkblue; background-color: lightyellow; padding: 10px;
border: 1px solid darkblue; border-radius: 5px;">
<h3>Tutorial</h3> Even if a statement or expression looks correct in terms of its syntax, it might still cause
an error when you try to run it. These errors, which happen while the program is running, are called exceptions.
Exceptions don’t necessarily stop your program completely; you can "catch" them and deal with them. However,
in most cases, when exceptions are not handled, the program stops and displays error messages.
Python comes with its own exceptions. But you can also define your own to make your code more manageable. This
is what we do below by defining "TotalUnimodularityError".
</div>

The function must raise an exception of type `TotalUnimodularityError` if the determinant is not 0, -1, or 1.
This exception is defined below.

In [ ]:


class TotalUnimodularityError(Exception):
    """Exception raised for errors in the total unimodularity check."""

    pass



We now implement the function calculating the determinant. Replace the `...`.

In [ ]:
def determinant_recursive(matrix: np.ndarray) -> float:
    """Calculates a determinant, and stops as soon as one sub-determinant is not 0, -1 or 1

    :param matrix: square matrix
    :return: determinant of the matrix

    :raise TotalUnimodularityError: if the determinant is not 0, -1 or 1.
    """
    # Check if the input is a NumPy ndarray
    if not isinstance(matrix, np.ndarray):
        raise ValueError('The input must be a NumPy ndarray.')

    # Check if the matrix is non-empty
    if matrix.size == 0:
        raise ValueError('The matrix must be non-empty.')

    # Check if the matrix is 2D
    if matrix.ndim != 2:
        raise ValueError('The matrix must be stored as a 2D numpy array.')

    rows, cols = matrix.shape

    if rows != cols:
        raise ValueError(f'The matrix must be square, and not {rows}x{cols}.')

    # If the matrix is 1x1, return the single element
    if rows == 1 and cols == 1:
        determinant = ...
        if check_valid_values(determinant):
            return determinant
        raise TotalUnimodularityError(f'Invalid element: {determinant}')

    # If the matrix is 2x2, return the determinant directly
    if rows == 2 and cols == 2:
        determinant = ...


        if check_valid_values(determinant):
            return determinant
        raise TotalUnimodularityError(
            f'Invalid determinant {determinant} for matrix\n{matrix}'
        )

    # Use cofactor expansion along the first row.
    determinant = 0.0
    for col in range(cols):
        element = float(matrix[0, col])
        if not check_valid_values(element):
            raise TotalUnimodularityError(f'Invalid element: {element}')
        submatrix = np.delete(np.delete(matrix, 0, axis=0), col, axis=1)
        cofactor = ...


        determinant += cofactor

    if check_valid_values(determinant):
        return determinant
    raise TotalUnimodularityError(
        f'Invalid determinant {determinant} for matrix\n{matrix}'
    )



We check the function.

In [ ]:
a_matrix = np.array([[1, 0, 0], [0, -1, 0], [0, 0, 1]])
print(a_matrix)



The function should not trigger any error.

In [ ]:
print(f'Determinant = {determinant_recursive(a_matrix)}')


In [ ]:
a_matrix = np.array([[2, 0, 0], [0, -1, 0], [0, 0, 1]])
print(a_matrix)


Here, an error should be triggered as one element is 2.

In [ ]:
try:
    value = determinant_recursive(a_matrix)
except TotalUnimodularityError as e:
    print(e)

In [ ]:
a_matrix = np.array([[1, 0, 0], [1, 1, 1], [-1, -1, 1]])
print(a_matrix)

Here, an error should be triggered as the determinant of a square submatrix is 2.

In [ ]:
try:
    value = determinant_recursive(a_matrix)
    print(value)
except TotalUnimodularityError as e:
    print(e)



Now, we can write a Python function that verifies if a matrix is totally unimodular.

In [ ]:
def is_totally_unimodular(matrix: np.ndarray) -> tuple[bool, str | None]:
    """Function that verifies if a matrix is totally unimodular.

    :param matrix: matrix to verify, a 2D numpy array.
    :return: if the matrix is totally unimodular, the tuple (True, None) is returned. If not, the tuple (False, msg)
        is returned, where the msg explains why it is not totally unimodular.
    """
    # Check if the input is a NumPy ndarray
    if not isinstance(matrix, np.ndarray):
        raise ValueError("The input must be a NumPy ndarray.")

    # Check if the matrix is non-empty
    if matrix.size == 0:
        raise ValueError("The matrix must be non-empty.")

    # Check if the matrix is 2D
    if matrix.ndim != 2:
        raise ValueError("The matrix must be stored as a 2D numpy array.")

    rows, cols = matrix.shape

    # We extract all square submatrices of maximum size, and check their determinant.
    max_size = min(matrix.shape)

    for i in range(rows - max_size + 1):
        for j in range(cols - max_size + 1):
            sub_matrix = matrix[i : i + max_size, j : j + max_size]
            ...





            return (
                False,
                f'Invalid determinant {determinant} for matrix\n{sub_matrix}',




We now test the function on the following matrices. Note that there may be several reasons
for not being totally unimodular, and the one reported here may not be the same as the one
reported for your implementation.


In [ ]:
A = np.array([[2, 1], [1, 4], [1, 1]])
print(A)

Expected result:<br>
`False`<br>
`Invalid determinant 7.0 for matrix`<br>
`[[2 1]`<br>
`[1 4]]`

In [ ]:
totally_unimodular, reason = is_totally_unimodular(matrix=A)
print(totally_unimodular)
if not totally_unimodular:
    print(reason)

In [ ]:
B = np.array([[0, 1, 0], [1, 0, 1]])
print(B)

Expected result:<br>
`True`

In [ ]:
totally_unimodular, reason = is_totally_unimodular(matrix=B)
print(totally_unimodular)
if not totally_unimodular:
    print(reason)

In [ ]:
C = np.array([[1, -1, 1], [1, 1, 0]])
print(C)

Expected result:<br>
`False`<br>
`Invalid determinant 2.0 for matrix`<br>
`[[ 1 -1]`<br>
`[ 1  1]]`

In [ ]:
totally_unimodular, reason = is_totally_unimodular(matrix=C)
print(totally_unimodular)
if not totally_unimodular:
    print(reason)